## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

#%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units

Import my library:

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)

import snapshot
import subhalo

In [ ]:
import importlib
importlib.reload(snapshot)
importlib.reload(subhalo)

# Rotation curves

Test rotation curve function and the identification of v_max, r_max and v_1kpc.

## Get data

In [ ]:
snap_id = 127
sim_ids = ["V1_LR_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix"]
names = ["plain-LCDM-LR", "curv-p082-LR", "curv-p082-LR"]

# Set directory for saving:
fig_path = os.path.abspath(os.path.join('..', 'Figures', 'LowResolution'))

# Which halo:
halo = [(1,0), (1,0), (1,0)]

# Set plot limits:
x_down = 0; x_up = 60
y_down = 0; y_up = 300

In [ ]:
snap_id = 127
sim_ids = ["V1_MR_fix", "V1_MR_curvaton_p082_fix"]
names = ["LCDM", "p082"]

# Set directory for saving:
fig_path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))

# Which halo:
halo = [(1,3), (1,4)]

# Set plot limits:
x_down = 0; x_up = 60
y_down = 0; y_up = 200

In [ ]:
data = {}
for name, sim_id, halo_id in zip(names, sim_ids, halo):
    data[name] = {"snapshot": snapshot.Snapshot(sim_id, snap_id, name=name),
                  "halo_identifier": halo_id}

In [ ]:
for name, sim_data in data.items():
    snap = sim_data["snapshot"]
    idx = snap.index_of_halo(sim_data["halo_identifier"][0], 
                             sim_data["halo_identifier"][1])

    rot_curves = snap.get_subhalos('Vcirc', 
                                   h5_group='Extended/RotationCurve/All')
    sub_offset = snap.get_subhalos('SubOffset', 
                                   h5_group='Extended/RotationCurve/All')
    v_circ = rot_curves[:,0] * units.cm.to(units.km)
    radii = rot_curves[:,1] * units.cm.to(units.kpc)
    
    max_point = snap.get_subhalos("Max_Vcirc", h5_group="Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    rmax = max_point[:,1] * units.cm.to(units.kpc)
    
    v1kpc = snap.get_subhalos("V1kpc", h5_group="Extended") * units.cm.to(units.km)
    
    data[name]["Vmax"] = vmax[idx]
    data[name]["Rmax"] = rmax[idx]
    data[name]["Vcirc"] = v_circ[sub_offset[idx]:sub_offset[idx+1]]
    data[name]["Radius"] = radii[sub_offset[idx]:sub_offset[idx+1]]
    data[name]["V1kpc"] = v1kpc[idx]

In [ ]:
color = ["black", "red", "blue", "green"]

In [ ]:
# Construct saving location:
filename = 'rotation_curve'
for name, sim_data in data.items():
    filename += "_{}_gn{}sgn{}".format(name, sim_data["halo_identifier"][0],
                                       sim_data["halo_identifier"][1])
filename += ".png"

filename = os.path.join(fig_path, filename)

In [ ]:
fig, axes = plt.subplots(figsize=(6,4))

# Set axes:
axes.set_xlim(x_down, x_up)
axes.set_ylim(y_down,y_up)

# Set labels:
axes.set_xlabel('$r[\mathrm{kpc}]$')
axes.set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$')

# Galaxy text:
#axes.set_title('Rotation curve')

for i, (name, entry) in enumerate(data.items()):
    lab = "{}: ({}, {})".format(name, 
                                entry["halo_identifier"][0],
                                entry["halo_identifier"][1])
    axes.plot(entry["Radius"], entry["Vcirc"], c=color[i], label=lab)
    axes.axhline(entry["Vmax"], c=color[i], linestyle='dashed')
    axes.axvline(entry["Rmax"], c=color[i], linestyle='dashed')
    axes.axhline(entry["V1kpc"], c=color[i], linestyle='dotted')
    
plt.legend()
plt.tight_layout()

# plt.savefig(os.path.join(path,filename), dpi=300)